# Eager Execution 튜토리얼: 기초

본 노트북은 텐서플로우의 eager execution의 기능을 소개하기 위한 기초 자료입니다. 다음과 같은 내용을 포함하고 있습니다:
* 필요한 패키지 불러오기
* eager execution 활성화
* TensorFlow 텐서와 변수를 만들고 사용하기 
* TensorFlow와 상호작용하며 사용하기
* eager execution 활성화 상태에서 GPU 사용하기 

본 노트북은 그레디언트와 같은 모델링 토픽은 다루고 있지 않습니다.

# Step 1: Eager 불러오기
eager execution을 위해서 다음과 같이 import 하세요:

In [1]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

Instructions for updating:
Use the retry module or similar alternatives.
TensorFlow version: 1.7.0
Eager execution: True


In [2]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))
print("")

x = tf.constant(2)
y = tf.constant(3)
print(x * y + 1)

# 대부분의 TensorFlow 연산은 eager execution에서 즉시 사용가능합니다.
# 다음과 같이 즉시 값을 반환해줍니다.
print(tf.contrib.signal.hamming_window(x * y + 1))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)

tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(
[0.08000001 0.31000003 0.77000004 1.         0.77       0.30999985
 0.08000001], shape=(7,), dtype=float32)


In [3]:
import numpy as np

ones = np.ones([3, 3])

print("1로 구성된 numpy 3x3행렬은:")
print(ones)
print("")

print("42를 곱하면:")
print(tf.multiply(ones, 42))

1로 구성된 numpy 3x3행렬은:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

42를 곱하면:
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)


In [4]:
x = tf.get_variable(name="x", shape=[], dtype=tf.float32, initializer=tf.zeros_initializer)

In [5]:
# 이 구문은 변수의 실제 값을 출력하지 않습니다:
print("Printing a TensorFlow Variable:")
print(x)
print("")

# TensorFlow 변수는 텐서에 대한 참조를 나타냅니다.
# `read_value()` 함수는 변수의 현재 값에 접근할 수 있도록 합니다. 
# Tensorflow 변수는 tf.get_variable()에 정의된대로 자동적으로 초기화 됩니다.
print("Printing a TensorFlow Variable's value using .read_value():")
print(x.read_value())
print("")

print("Printing a TensorFlow Variable's value using .read_value().numpy():")
print(x.read_value().numpy())

Printing a TensorFlow Variable:
<tf.Variable 'x:0' shape=() dtype=float32, numpy=0.0>

Printing a TensorFlow Variable's value using .read_value():
tf.Tensor(0.0, shape=(), dtype=float32)

Printing a TensorFlow Variable's value using .read_value().numpy():
0.0


In [6]:
x.assign(42)
print(x.read_value())

x.assign_add(3)
print(x.read_value())

tf.Tensor(42.0, shape=(), dtype=float32)
tf.Tensor(45.0, shape=(), dtype=float32)


In [7]:
print(x + 3)

# 이 코드는 숫자의 리스트에 대해 자동으로 브로드캐스팅 됩니다.
print(x * [1, 2, 4])

tf.Tensor(48.0, shape=(), dtype=float32)
tf.Tensor([ 45.  90. 180.], shape=(3,), dtype=float32)


In [8]:
vector = tf.constant([10.0, 20.0, 30.0, 40.0])

In [9]:
# 2번째, 3번째 인자로 전달된 `begin`과 `size`가 `vector`의 범위 내에 포함되어서 잘 동작합니다.
print(tf.slice(vector, [1], [3]))

tf.Tensor([20. 30. 40.], shape=(3,), dtype=float32)


In [10]:
# 이 코드는 동작하지 않습니다.
# 3번째 인자인 `size`가 `vector`의 범위를 넘어서는 인덱스를 요청했기 때문입니다.
# 에러는 즉시 발생합니다.
try:
  print(tf.slice(vector, [1], [4]))
except tf.OpError as e:
  print("Caught error: %s" % e)

Caught error: Expected size[0] in [0, 3], but got 4 [Op:Slice]


In [11]:
# 본 예제 코드는 사용자의 노트북이 CUDA GPU 환경에서 동작 할 경우에만 작동합니다.
# 아래 구문이 해당 사항을 체크합니다.
is_gpu_available = tfe.num_gpus() > 0

# 임의의 Tensors를 만듭니다. 
SIZE = 1000
cpu_tensor = tf.random_normal([SIZE, SIZE])

if is_gpu_available:
  gpu_tensor = cpu_tensor.gpu()
else:
  print("GPU not available.")

In [12]:
# CPU 기반 행렬 곱셈 시간 측정

print("CPU에서 matmul에 걸리는 시간:")
%time tf.matmul(cpu_tensor, cpu_tensor)

CPU에서 matmul에 걸리는 시간:
CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 43 ms


<tf.Tensor: id=96, shape=(1000, 1000), dtype=float32, numpy=
array([[   7.084673 ,   56.023304 , -109.85308  , ...,    0.858407 ,
         -33.955086 ,   14.23424  ],
       [  -5.7779913,   13.537379 ,  -65.76546  , ...,  -13.514355 ,
         -21.036415 ,  -23.941998 ],
       [ -24.80651  ,   11.101314 ,   29.67744  , ...,   29.810043 ,
          49.076576 ,   16.461351 ],
       ...,
       [ -22.55507  ,   11.751124 ,   26.205606 , ...,  -10.737196 ,
         -25.52725  ,   39.644978 ],
       [ -13.076582 ,  -14.075956 ,  -14.266737 , ...,    3.357493 ,
         -27.851429 ,  -26.955614 ],
       [   9.08893  ,   24.286556 ,   12.602869 , ...,   -5.659523 ,
         -51.025173 ,  -12.490817 ]], dtype=float32)>

In [13]:
# GPU 기반 행렬 곱셈 시간 측정

if is_gpu_available:
  # 처음엔 GPU 초기화 때문에 시간이 걸립니다. :
  print("GPU에서 첫번째 matmul에 걸리는 시간:")
  %time tf.matmul(gpu_tensor, gpu_tensor)
  print()

  # 연속적으로 사용할 경우 훨씬 빠릅니다.:
  print("GPU에서 두번째 matmul에 걸리는 시간:")
  %time tf.matmul(gpu_tensor, gpu_tensor)

GPU에서 첫번째 matmul에 걸리는 시간:
CPU times: user 112 ms, sys: 64 ms, total: 176 ms
Wall time: 604 ms

GPU에서 두번째 matmul에 걸리는 시간:
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 527 µs


In [14]:
# 두번째 시간을 GPU를 위한 데모로 측정, 첫번째는 초기화 과정으로 통과 시킵니다:

cpu_tensor = tf.random_normal([SIZE, SIZE])
print("CPU에서 matmul에 걸리는 시간:")
%time tf.matmul(cpu_tensor, cpu_tensor)
print()

if is_gpu_available:
  gpu_tensor = cpu_tensor.gpu()
  print("GPU에서 matmul에 걸리는 시간:")
  %time tf.matmul(gpu_tensor, gpu_tensor)

CPU에서 matmul에 걸리는 시간:
CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 26.7 ms

GPU에서 matmul에 걸리는 시간:
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 580 µs
